In [ ]:
#!/usr/bin/env python

# Copyright (C) 2017 Electric Movement Inc.
#
# This file is part of Robotic Arm: Pick and Place project for Udacity
# Robotics nano-degree program
#
# All Rights Reserved.

# Author: Harsh Pandya

# import modules
import rospy
import tf
import numpy as np
from numpy import array
from kuka_arm.srv import *
from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
from geometry_msgs.msg import Pose
from mpmath import *
from sympy import *
from sympy import symbols, cos, sin, pi, simplify, sqrt, atan2, acos
from sympy.matrices import Matrix



def handle_calculate_IK(req):
    rospy.loginfo("Received %s eef-poses from the plan" % len(req.poses))
    if len(req.poses) < 1:
        print "No valid poses received"
        return -1
    else:
        # Initialize service response
        joint_trajectory_list = []
        for x in xrange(0, len(req.poses)):
            # IK code starts here
            joint_trajectory_point = JointTrajectoryPoint()

            # Define DH param symbols - my code start here:



	    q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')
	    d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
	    a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
	    alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')


	    s = {alpha0:      0,   a0:     0, d1: 0.75,     q1: 0,   #the base is fixed to the ground
                 alpha1:  -pi/2,   a1:  0.35, d2:     0,    q2:  -np.pi/2,
	         alpha2:      0,   a2:  1.25, d3:     0,    q3:  0,
	         alpha3:  -pi/2,   a3:-0.054, d4:  1.50,    q4:  0,
	         alpha4:   pi/2,   a4:     0, d5:     0,    q5:  0,
	         alpha5:  -pi/2,   a5:     0, d6:     0,    q6:  0,
	         alpha6:      0,   a6:     0, d7: 0.303,    q7: 0}

	    T0_1 = Matrix([[             cos(q1),            -sin(q1),             0,              a0],
                                [ sin(q1)*cos(alpha0), cos(q1)*cos(alpha0),  -sin(alpha0),  -sin(alpha0)*d1],
 		                [ sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),   cos(alpha0),   cos(alpha0)*d1],
		                [                   0,                   0,             0,                1]])

	    T0_1 = T0_1.subs(s)

	    T1_2 = Matrix([[             cos(q2),            -sin(q2),             0,              a1],
              [ sin(q2)*cos(alpha1), cos(q2)*cos(alpha1),  -sin(alpha1),  -sin(alpha1)*d2],
              [ sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),   cos(alpha1),   cos(alpha1)*d2],
              [                   0,                   0,             0,                1]])

	    T1_2 = T1_2.subs(s)

	    T2_3 = Matrix([[             cos(q3),            -sin(q3),             0,              a2],
              [ sin(q3)*cos(alpha2), cos(q3)*cos(alpha2),  -sin(alpha2),  -sin(alpha2)*d3],
              [ sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),   cos(alpha2),   cos(alpha2)*d3],
              [                   0,                   0,             0,                1]])

	    T2_3 = T2_3.subs(s)

	    T3_4 = Matrix([[             cos(q4),            -sin(q4),             0,              a3],
              [ sin(q4)*cos(alpha3), cos(q4)*cos(alpha3),  -sin(alpha3),  -sin(alpha3)*d4],
              [ sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),   cos(alpha3),   cos(alpha3)*d4],
              [                   0,                   0,             0,                1]])

	    T3_4 = T3_4.subs(s)

	    T4_5 = Matrix([[             cos(q5),            -sin(q5),             0,              a4],
              [ sin(q5)*cos(alpha4), cos(q5)*cos(alpha4),  -sin(alpha4),  -sin(alpha4)*d5],
              [ sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),   cos(alpha4),   cos(alpha4)*d5],
              [                   0,                   0,             0,                1]])

	    T4_5 = T4_5.subs(s)

	    T5_6 = Matrix([[             cos(q6),            -sin(q6),             0,              a5],
              [ sin(q6)*cos(alpha5), cos(q6)*cos(alpha5),  -sin(alpha5),  -sin(alpha5)*d6],
              [ sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),   cos(alpha5),   cos(alpha5)*d6],
              [                   0,                   0,             0,                1]])

	    T5_6 = T5_6.subs(s)

	    T6_G = Matrix([[             cos(q7),            -sin(q7),             0,              a6],
              [ sin(q7)*cos(alpha6), cos(q7)*cos(alpha6),  -sin(alpha6),  -sin(alpha6)*d7],
              [ sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),   cos(alpha6),   cos(alpha6)*d7],
              [                   0,                   0,             0,                1]])

	    T6_G = T6_G.subs(s)

	    T0_2 = simplify(T0_1*T1_2)
	    T0_3 = simplify(T0_2*T2_3)
	    T0_4 = simplify(T0_3*T3_4)
	    T0_5 = simplify(T0_4*T4_5)
	    T0_6 = simplify(T0_5*T5_6)
	    T0_G = simplify(T0_6*T6_G)

	    R_z = Matrix([[    cos(np.pi),   -sin(np.pi),      0,       0],
              [    sin(np.pi),    cos(np.pi),      0,       0],
              [             0,             0,      1,       0],
              [             0,             0,      0,       1]])

	    R_y = Matrix([[ cos(-np.pi/2),             0, sin(-np.pi/2),    0],
              [             0,             1,             0,    0],
              [-sin(-np.pi/2),             0, cos(-np.pi/2),    0],
              [             0,             0,             0,    1]])

	    R_corr = simplify(R_z * R_y)

	    # Can insert some error measuring codes here
	    T_total = simplify(T0_G*R_corr)
 	    #Here we are working on the R6_0 matrix which rotates the 6 Frame to be 
	    #aligned with the 0 Frame.  It should be the transpose of the yaw*pitch*roll
	    #matrix, which is T(roll)*T(pitch)*T(yaw)as below: (assumes these are in radians)

	    #yaw, pitch, and roll are returned by the program

	    #moved code up from to define yaw, pitch, roll, px, py, pz:
	    # roll, pitch, yaw = end-effector orientation
            px = req.poses[x].position.x
            py = req.poses[x].position.y
            pz = req.poses[x].position.z

            (roll, pitch, yaw) = tf.transformations.euler_from_quaternion(
                [req.poses[x].orientation.x, req.poses[x].orientation.y,
                    req.poses[x].orientation.z, req.poses[x].orientation.w])


	    Y = Matrix([[cos(yaw),  -sin(yaw),    0],
            [sin(yaw),   cos(yaw),    0],
            [       0,          0,    1]])


	    P = Matrix([[cos(pitch),        0,    sin(pitch)],
            [         0,        1,    0],
            [-sin(pitch),       0,    cos(pitch)]])

	    R = Matrix([[         1,        0,     0],
            [         0,cos(roll),    -sin(roll)],
            [         0,sin(roll),     cos(roll)]])

            #R0_6 = simplify(R*P*Y*R_corr[0:3, 0:3])
	    #R6_0 = R0_6.T
            #R6_0 = Y*P*R*R_corr[0:3, 0:3].T
	    R6_0 = (Y*P*R)*R_corr[0:3, 0:3].T
	    #R6_0 = R6_0*R_corr.T
	    # Also could have said R6_0 = Y*P*R, and R0_6 = R6_0.T
	    #Here we calculate the vector 0rWC/0, or the vector from the origin (base
	    #of robot) to the WC (which is joint 5).  Program returns px, py, pz.
	    #d = .0375 (length from J6 to ee) + .193 (length from J5 to J6)

	    #moved this code upfront to define px, py, pz

	    d = .303    #updated this from .0375 + .193
	    rwc0 = Matrix([px,  py,  pz]) - d*R6_0*Matrix([0, 0, 1])
	    rwc0x = rwc0[0, 0]
	    rwc0y = rwc0[1, 0]
	    rwc0z = rwc0[2, 0]

	    # Here we calculate the length of Link3 - this only needs to be done once
	    # so it should be moved up higher

	    L3 = sqrt(.054**2 + 1.5**2)

	    #List lengths of all links for trigonometry calculations
	    L2 = 1.25
	    L34 = 1.5  #approximately - had this at 2.04 earlier

	    #calculate rotation at joint 1, angles

	    theta1 = atan2(rwc0y, rwc0x)

	    #This moves everything into the x,z plane, so that rwc0y == 0 now.
	    #  But this will change the x and z coordinates with a rotation around the
	    # z-axis of the origin by theta1.  So must recalculate the 
	    # relative coordinates of the rwc0 using -theta1 because it is like doing a
	    # clockwise rotation around the fixed coordinates:


	    Zrot = Matrix([[cos(-theta1),  -sin(-theta1),    0],
            [sin(-theta1),   cos(-theta1),    0],
            [       0,          0,    1]])

	    rwc0new = Zrot*rwc0

	    # and now we are only interested in the x, z coordinates because only these
	    # determine the remaining angles:

	    rwc0newx = rwc0new[0, 0]
	    rwc0newz = rwc0new[2, 0]

	    #need to rename for convenience

	    newx = rwc0newx
	    newz = rwc0newz

	    #the location of Joint2 in (x, z) is (.35, .75) according URDF.  We now
	    # use the distance formula to obtain the distance between J2 and WC:

	    dst1 = sqrt((newx - .35)**2 + (newz - .75)**2)

	    #Since there must be a triangle with sides L2, L3, and dst1 we can use the
	    #law of cosines to determine the needed angle at Joint3:

	    theta3 = acos((dst1**2 - L2**2 - L34**2)/(-2*L2*L34))

	    #Because of the configuration of the zero angle, must make an adjustment to theta3
	    # The negative sign is because of the position of the Z axis
            #actually, I think the negative has got to go
	    theta3 = -(theta3 - np.pi/2)

	    #Now on to figuring out theta2.  There must be a triangle with sides
	    #0.35, dst1, and third side with the magnitude of the distance between
	    #joint 2 and joint 4.  This triangle has vertices at joints 1, 2, and 4
	    #therefore  2*pi minus the angle at joint 2 gives us theta2:

	    #first figure out distance between joints 1 and 4:
	    dst2 = sqrt((newx)**2 + (newz - .75)**2)

	    #Now use law of cosines again:
	    #theta2 = acos(((dst2)**2 - (.35)**2 - (dst1)**2)/((-2)*(.35)*(dst1)))
            theta2 = atan2(newz - 0.75, newx - 0.35)
	    thetatemp = acos((L34**2 - L2**2 - dst1**2)/(-2*L2*dst1))
	    theta2 = theta2 + thetatemp 
            #theta2 = -(theta2 - np.pi/2)  based on new info take this line out
            theta2 = -theta2
	    theta2 = theta2 + np.pi/2
	    #Again because of the positioning we must make an adjustment to theta2:
            #Actually I think we have to leave theta2 alone   
	    #theta2 = np.pi/2 - theta2
	    #Now with theta1, theta2, and theta3 we need to use forward kinematics to determine the value
	    # of R3_0:
    
	    #s = {q1: 0, q2: theta1, q3: theta2, q4: theta3}

	    s = {alpha0:      0,   a0:     0, d1:  0.75,    q1: theta1,   #the base is fixed to the ground
	     alpha1:  -pi/2,   a1:  0.35, d2:     0,    q2: theta2 - np.pi/2,  #was q2:  q2-pi/2
	     alpha2:      0,   a2:  1.25, d3:     0,    q3: theta3
    		 }


	    T0_1 = Matrix([[             cos(q1),            -sin(q1),             0,              a0],
              [ sin(q1)*cos(alpha0), cos(q1)*cos(alpha0),  -sin(alpha0),  -sin(alpha0)*d1],
              [ sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),   cos(alpha0),   cos(alpha0)*d1],
              [                   0,                   0,             0,                1]])

	    T0_1 = T0_1.subs(s)

	    T1_2 = Matrix([[             cos(q2),            -sin(q2),             0,              a1],
              [ sin(q2)*cos(alpha1), cos(q2)*cos(alpha1),  -sin(alpha1),  -sin(alpha1)*d2],
              [ sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),   cos(alpha1),   cos(alpha1)*d2],
              [                   0,                   0,             0,                1]])

	    T1_2 = T1_2.subs(s)

	    T2_3 = Matrix([[             cos(q3),            -sin(q3),             0,              a2],
              [ sin(q3)*cos(alpha2), cos(q3)*cos(alpha2),  -sin(alpha2),  -sin(alpha2)*d3],
              [ sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),   cos(alpha2),   cos(alpha2)*d3],
              [                   0,                   0,             0,                1]])

	    T2_3 = T2_3.subs(s)

	    #T3_4 = Matrix([[             cos(q4),            -sin(q4),             0,              a3],
	    #             [ sin(q4)*cos(alpha3), cos(q4)*cos(alpha3),  -sin(alpha3),  -sin(alpha3)*d4],
 	    #            [ sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),   cos(alpha3),   cos(alpha3)*d4],
	    #           [                   0,                   0,             0,                1]])

	    #T3_4 = T3_4.subs(s)

	    T0_2 = simplify(T0_1*T1_2)
	    T0_3 = simplify(T0_2*T2_3)
	    #T0_4 = simplify(T0_3*T3_4)

	    #First Need to slice T0_3 in order to get R0_3, then can
	    #calculate R3_6:


	    R3_0 = T0_3[0:3, 0:3]
	    #R0_3=T0_3[0:3, 0:3]
	    R6_3 = R3_0.T*R6_0 #this should be rdone as below:
	    #R6_3=R0_3*R6_0
	    

	    #Convert sympy slices to indices

	    r_13 = R6_3[0,2]
	    r_21 = R6_3[1,0]
	    r_22 = R6_3[1,1]
	    r_23 = R6_3[1,2]
	    r_33 = R6_3[2,2]

	    #But R3_6 is an intrinsic Y-Z-Y rotation using Link 3 as the baseline
	    #coordinate system.  We then must make the appropriate equalities according to the
	    #printout for the product Y1*Z1*Y2

	    theta4 = atan2(r_33, -r_13)
	    theta5 = atan2(sqrt(r_21**2+r_22**2), r_23)
            theta5.evalf()
	    #theta5 = acos(r_23)
	    theta6 = atan2(-r_22, r_21)
            theta6.evalf()





            print("HERE" + str(theta1) + str(theta2) + str(theta3) + str(theta4) + str(theta5) + str(theta6))

            
            # Joint angle symbols


      
            # Modified DH params


            
            # Define Modified DH Transformation matrix



            # Create individual transformation matrices


            
            # Extract end-effector position and orientation from request
	    # px,py,pz = end-effector position
	    # roll, pitch, yaw = end-effector orientation
            #px = req.poses[x].position.x
            #py = req.poses[x].position.y
            #pz = req.poses[x].position.z

            #(roll, pitch, yaw) = tf.transformations.euler_from_quaternion(
            #    [req.poses[x].orientation.x, req.poses[x].orientation.y,
            #        req.poses[x].orientation.z, req.poses[x].orientation.w])
     
            # Calculate joint angles using Geometric IK method

		


            # Populate response for the IK request
            # In the next line replace theta1,theta2...,theta6 by your joint angle variables
	    joint_trajectory_point.positions = [theta1, theta2, theta3, theta4, theta5, theta6]
	    joint_trajectory_list.append(joint_trajectory_point)

        rospy.loginfo("length of Joint Trajectory List: %s" % len(joint_trajectory_list))
        return CalculateIKResponse(joint_trajectory_list)


def IK_server():
    # initialize node and declare calculate_ik service
    rospy.init_node('IK_server')
    s = rospy.Service('calculate_ik', CalculateIK, handle_calculate_IK)
    print "Ready to receive an IK request"
    rospy.spin()

if __name__ == "__main__":
    IK_server()
robond@udacity:~/catkin_ws/src/RoboND-Kinematics-Project/kuka_arm/scripts$ 
